In [1]:
import os
import librosa 
import json 

data = {"genres" : [], "mfccs": [],"zrc":[], "labels" : [] }

folder = r"PATH_TO_DATA"
filepaths = []
cur_label = -1
num_seg = 5
json_path = "data.json"

SAMPLE_RATE = 22050
AUDIO_TIME = 30
SAMPLES_PER_SONG = int(SAMPLE_RATE * AUDIO_TIME) 
SAMPLES_PER_SEGMENT = int(SAMPLES_PER_SONG / num_seg)

# Go through all the folders
for subdir, dirs, files in os.walk(folder):
    for file in files:
        if file.lower().endswith('.mp3'):
            cur_genre = subdir.split("\\")[-1]
            
            # Save the name of the cur genre in the genres 
            if subdir.split("\\")[-1] not in data["genres"]:
                data["genres"].append(cur_genre)
                cur_label += 1
                
            # Get the mfcc for the current file 
            audio,sr = librosa.load(os.path.join(subdir, file),sr = SAMPLE_RATE)
            
            for i in range(num_seg):
                start = (sr * 30) + (i * SAMPLES_PER_SEGMENT)
                end = start + SAMPLES_PER_SEGMENT
                cur_mfcc = librosa.feature.mfcc(y = audio[start:end],
                                               sr = sr,
                                               n_mfcc = 13,
                                               n_fft = 2048,
                                               hop_length = 512)
                
                
                zcr = librosa.feature.zero_crossing_rate(audio[start:end]).tolist()

                #Store the MFCC of the audio segment 
                data["mfccs"].append(cur_mfcc.T.tolist())
                data["zrc"].append(zcr)
                data["labels"].append(cur_label)
# Save json file            
with open(json_path, "w") as json_file:
    json.dump(data,json_file, indent = 4)